In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.guzinski import * 
import time
from joblib import Parallel, delayed
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.mirmazloumi import *
import numpy as np
from osgeo import gdal, osr
from datetime import datetime, timezone
import re
import time
import pandas as pd
import pvlib
from pvlib import solarposition
from pvlib.irradiance import get_total_irradiance
from pvlib.location import Location
from scipy.interpolate import NearestNDInterpolator
from other_repos.pyDMS.pyDMS.pyDMS import *
from other_repos.pyTSEB.pyTSEB import meteo_utils
from other_repos.pyTSEB.pyTSEB import resistances
from other_repos.pyTSEB.pyTSEB import net_radiation
from other_repos.pyTSEB.pyTSEB import clumping_index 
from other_repos.pyTSEB.pyTSEB import TSEB

In [6]:

# paths
lowmask_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP.tif'
lowmask_bin_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP_BINARY.tif'
temp_dump = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/SharpEvap/Brandenburg/SecondShot/'
trash_path = f'{temp_dump}trash/'

path_to_slope = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/SLOPE/'
path_to_aspect = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/ASPECT/'
path_to_agro = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/THUENEN_2021/'

path_to_dem = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/DEM/'
path_to_lat = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/LAT/'
path_to_lon = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/LON/'
path_to_acq = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/'
path_to_vza = '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/comp/'
path_to_vaa = '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/comp/'

path_to_force = '/data/Aldhani/eoagritwin/force/output/Guzinski'
path_to_inci = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/INCIDENCE2/'
path_to_lst = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/LST_composites/'

# get the Tiles for Brandenburg
bran = pd.read_csv('/data/Aldhani/eoagritwin/misc/state_tile_csv/clipped_grid_bran_tiles.csv')
tiles_to_process = createFORCEtileLIST(list(bran['Tile_X']),
                                        list(bran['Tile_Y']), True)


In [8]:
len(tiles_to_process)


52

In [2]:
# paths
lowmask_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP.tif'
lowmask_bin_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP_BINARY.tif'
temp_dump = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/SharpEvap/Brandenburg/IntBio/'
trash_path = f'{temp_dump}trash/'

path_to_slope = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/SLOPE/'
path_to_aspect = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/ASPECT/'
path_to_agro = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/THUENEN_2021/'

path_to_dem = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/DEM/'
path_to_lat = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/LAT/'
path_to_lon = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/LON/'
path_to_acq = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/'
path_to_vza = '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/comp/'
path_to_vaa = '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/comp/'

path_to_force = '/data/Aldhani/eoagritwin/force/output/Guzinski'
path_to_inci = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/INCIDENCE2/'
path_to_lst = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/LST_composites/'

# get the Tiles for Brandenburg
bran = pd.read_csv('/data/Aldhani/eoagritwin/misc/state_tile_csv/clipped_grid_bran_tiles.csv')
tiles_to_process = createFORCEtileLIST(list(bran['Tile_X']),
                                        list(bran['Tile_Y']), True)

tiles_to_process.sort()
joblist = []


tile_to_process = tiles_to_process[-47]
storFolder = temp_dump
path_to_slope
path_to_aspect
path_to_agro
path_to_force
path_to_inci
path_to_lst
time_start = '20190401'
time_end = '20190420'
compList = ['maxLST']
predList = ['S2only']
S2mask = 3
printEvapInter=False


temp_dump_fold = path_safe(f"{storFolder}temp/{tile_to_process.replace('_', '')}/")
sharp_outFolder = path_safe(f"{storFolder}sharpened/{tile_to_process.replace('_', '')}/")
evap_outFolder = path_safe(f"{storFolder}evap/{tile_to_process.replace('_', '')}/")
trash_path = path_safe(f"{storFolder}trash/")


year = time_start[:4]

# ############## make vrts for slope, aspect and agromask
slopes = [file for file in getFilelist(path_to_slope, '.tif') if tile_to_process in file] # if any tile name is in file
# aspect-tiles
aspects = [file for file in getFilelist(path_to_aspect, '.tif') if tile_to_process in file] # if any tile name is in file
# thuenen-tiles
thuenen = [file for file in getFilelist(path_to_agro, '.tif') if tile_to_process in file] # if any tile name is in file

dems = [file for file in getFilelist(path_to_dem, '.tif') if tile_to_process in file] # if any tile name is in file
lats = [file for file in getFilelist(path_to_lat, '.tif') if tile_to_process in file] # if any tile name is in file
lons = [file for file in getFilelist(path_to_lon, '.tif') if tile_to_process in file] # if any tile name is in file


# get those tiles (and composite if more than one tile is provided)
slope_path = f'{temp_dump_fold}SLOPE.vrt'
gdal.BuildVRT(slope_path, slopes)

aspect_path = f'{temp_dump_fold}ASPECT.vrt'
gdal.BuildVRT(aspect_path, aspects)

thuenen_path = f'{temp_dump_fold}THUENEN.vrt'
gdal.BuildVRT(thuenen_path, thuenen)

dem_path = f'{temp_dump_fold}DEM.vrt'
gdal.BuildVRT(dem_path, dems)

lat_path = f'{temp_dump_fold}LAT.vrt'
gdal.BuildVRT(lat_path, lats)
              
lon_path = f'{temp_dump_fold}LON.vrt'
gdal.BuildVRT(lon_path, lons)


# ################ load force and vrt
path_to_S2_tiles = f'{path_to_force}/{year}/'
# get a list with all available tiles
files = getFilelist(f'{path_to_S2_tiles}', '.tif', deep=True) 
files = [file for file in files if tile_to_process in file]
date_list = check_forceTSI_compositionDates(files)

# make the mask ready for S2 masking
th_ds = gdal.Open(thuenen_path)
th_arr = th_ds.GetRasterBand(1).ReadAsArray()
mask = np.where(th_arr == -9999, 0, 1)

colors = ['BLU', 'GRN', 'RED', 'BNR', 'NIR', 'RE1', 'RE2', 'RE3',  'SW1', 'SW2']

dateList_for_csv = []
# this should be the masterloop within the sharpend, evaping and deleting of all files but the above created vrts takes place
for date in date_list:

    if int(time_start) <= int(date) <= int(time_end): # neede to be adapted in a manner that incorporates the +/-4 otherwise time_start 20190401 
        # would find date 20190406 as int(time_start) <= int(date) <= int(time_end)..

        dateList_for_csv.append(date)

        lowRes_files = []
        highRes_files = []
        highRes_names = []

        # get S2 bands
        tilesS2 = [file for file in getFilelist(path_to_S2_tiles, '.tif', deep=True) if tile_to_process in file and f'{date}.tif' in file]
        tilesS2 = [t2 for col in colors for t2 in tilesS2 if col in t2]
        S2_path = f'{temp_dump_fold}S2_{date}.vrt'

        # build vrt
        vrt = gdal.BuildVRT(S2_path, tilesS2, separate=True)
        vrt = None

        vrt = gdal.Open(S2_path, gdal.GA_Update)  # VRT must be writable
        for idz, bname in enumerate(colors): 
            band = vrt.GetRasterBand(1+idz)
            band.SetDescription(bname)
        vrt = None

        # determine LST and incidence files associated with respective S2 composite
        band_dict = transform_compositeDate_into_LSTbands(date, 4)

        # stat used for compositing
        for comp_stat in compList: #  
            path_to_incident = f'{path_to_inci}{comp_stat}/{year}/'
            path_to_LST = f'{path_to_lst}{comp_stat}/{year}/'
            print(path_to_LST)

            for k, v in band_dict.items():
                month = v['month']
                band = int(v['band'])
                v_path = f'{path_to_LST}Daily_LST_{comp_stat}_{year}_{month}.tif'
                ds = gdal.Open(v_path, 0)

                # calculate biophysical helper and parameter
                calc_biophys_helper(acq_path=path_to_acq, dem_path=dem_path, lat_path=lat_path, lon_path=lon_path, vaa_path=path_to_vaa,
                                    vza_path=path_to_vza, year=year, month=month, doy=band, outPath=temp_dump_fold, comp_stat=comp_stat)

                get_biophysical_parameter(path_S2=tilesS2, path_Sun=f"{temp_dump_fold}sun/", year=year, month=month, doy=band, outPath=temp_dump_fold)
                
                
                # export the LST for that day
                LST_arr = ds.GetRasterBand(band).ReadAsArray() # store as single Tiff in temp
                daily_lst_path = f'{temp_dump_fold}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif'
                makeTif_np_to_matching_tif(LST_arr, v_path, daily_lst_path)

                # store the paths for selecting incidence for corresponding LST
                incid_date = f'{year}_{month}_{band:02d}.tif'

                # incidence-tiles
                #incids = [file for file in getFilelist(path_to_incident, '.tif', deep=True) if tile_to_process in file] 
                incid_path = [file for file in getFilelist(path_to_incident, '.tif', deep=True) if f'{tile_to_process}_{incid_date}' in file][0]

                # create highRes file through exapnding the vrt of S2
                highRes_path = f"{temp_dump_fold}HIGHRES_{comp_stat}_{incid_date.split('.')[0]}.vrt"
                gdal.BuildVRT(highRes_path, [S2_path, slope_path, aspect_path, incid_path], separate=True)

                for predi in predList:
                    if predi == 'allpred':
                        maskVRT_water(highRes_path, colorlist=colors)
                    else:
                        maskVRT_water_and_drop_aux(highRes_path, colorlist=colors)

                    if S2mask == 1:
                        highRes_files.append(f"{highRes_path.split('.')[0]}_watermask.tif")
                        highRes_names.append(f"S2notMasked_{predi}")
                        lowRes_files.append(daily_lst_path)

                    elif S2mask == 2:
                        maskVRT(f"{highRes_path.split('.')[0]}_watermask.tif", mask, suffix=f"_S2_agromask_{predi}")
                        os.remove(f"{highRes_path.split('.')[0]}_watermask.tif")
                        highRes_files.append(f"{highRes_path.split('.')[0]}_watermask_S2_agromask_{predi}.tif")
                        lowRes_files.append(daily_lst_path)
                        highRes_names.append(f"S2Masked_{predi}")

                    elif S2mask == 3:
                        highRes_files.append(f"{highRes_path.split('.')[0]}_watermask.tif")
                        highRes_names.append(f"S2notMasked_{predi}")
                        lowRes_files.append(daily_lst_path)
                        maskVRT(f"{highRes_path.split('.')[0]}_watermask.tif", mask, suffix=f"_S2_agromask_{predi}")
                        highRes_files.append(f"{highRes_path.split('.')[0]}_watermask_S2_agromask_{predi}.tif")
                        lowRes_files.append(daily_lst_path)
                        highRes_names.append(f"S2Masked_{predi}")

                
            # sharp all files for that day and all compstats
            sharpList = []

            for idx, highResFilename in enumerate(highRes_files):
                    lowResFilename = lowRes_files[idx]
                    # f1 = f'{sharp_outFolder}{'/'.join(highResFilename.split('.')[0].split('_')[2:5])}/'
                    for maskname, mask_lowRes in zip(['withoutLSTmask'], ['']): # 'withLSTmask'  lowmask_bin_path
                    # for maskname, mask_lowRes in zip(['withoutLSTmask', 'withLSTmask'], ['', lowmask_bin_path]):
                        lowResMaskFilename = mask_lowRes
                        # f2 = f'{f1}{maskname}/'
                        for movWin in [15]:
                            for cv in [0]:
                                for regrat in [0.25]:
                                    kombi = f'mvwin{movWin}_cv{cv}_regrat{int(regrat*100):02d}_{highRes_names[idx]}_{maskname}'
                                    # f3 = f'{f2}{highRes_names[idx]}/'
                                    # os.makedirs(f'{f3}Residuals/', exist_ok=True)
                                    # os.makedirs(f'{f3}Values/', exist_ok=True)

                                    os.makedirs(f'{sharp_outFolder}Residuals/', exist_ok=True)
                                    os.makedirs(f'{sharp_outFolder}Values/', exist_ok=True)
                                    
                                    # sharpened_file = f'{f3}{'_'.join(highResFilename.split('.')[0].split('_')[2:6])}_{kombi}_{tile_to_process}.tif'
                                    sharpened_file = f"{sharp_outFolder}{'_'.join(highResFilename.split('.')[0].split('_')[1:5])}_{kombi}_{tile_to_process}.tif"
                                    
                                    runSharpi(highResFilename, lowResFilename, lowResMaskFilename, cv, movWin, regrat, sharpened_file, useDecisionTree = True)

                                    # run the biophysical parameter calculation here
                                    
                                    # should run the function and return a list with the paths for all the created files
                                    # this list will then be passed on to runEvapi
                                    sharpList.append(sharpened_file)
            # calc evapo for all compstats at that day
            for sharped in sharpList:
                comp = sharped.split('/')[-1].split('_')[0]
                year = sharped.split('/')[-1].split('_')[1]
                month = sharped.split('/')[-1].split('_')[2]
                day = int(sharped.split('/')[-1].split('_')[3])
                mvwin = sharped.split('/')[-1].split('_')[4]
                cv = sharped.split('/')[-1].split('_')[5]
                regrat = sharped.split('/')[-1].split('_')[6]
                sharp = sharped.split('/')[-1].split('_')[8]
                s2Mask = sharped.split('/')[-1].split('_')[7]
                lstMask = sharped.split('/')[-1].split('_')[9]
                tile = '_'.join(sharped.split('/')[-1].split('.')[0].split('_')[-2:])

                if path_to_dem:
                    bio_pars = [file for file in getFilelist(f'{temp_dump_fold}bio/', '.tif') if f'{year}_{month}_{day}.tif' in file]
                else:
                    bio_pars = False

                runEvapi(year=year, month=month, day=day, comp=comp, sharp=sharp, s2Mask=s2Mask, lstMask=lstMask, tile=tile,
                        tempDir=trash_path, path_to_temp=temp_dump_fold, path_to_sharp=sharp_outFolder, mvwin=mvwin, cv=cv,
                        regrat=regrat, evap_outFolder=evap_outFolder, printInterim=printEvapInter, bio=bio_pars)
            
            
            # at the end of the date loop --> clean up temp folder and sharp
            temp_files_vrt = [file for file in getFilelist(temp_dump_fold, '.vrt')]
            temp_files_tif = [file for file in getFilelist(temp_dump_fold, '.tif')]
            sharp_files = [file for file in getFilelist(sharp_outFolder, '.tif', deep=True)]
            [os.remove(file) for file in temp_files_vrt]
            [os.remove(file) for file in temp_files_tif]
            [os.remove(file) for file in sharp_files]
            if path_to_dem:
                [temp_files_vrt.remove(path) for path in [slope_path, aspect_path, thuenen_path, dem_path, lat_path, lon_path]]
                bio_files = [file for file in getFilelist(f'{temp_dump_fold}bio/', '.tif')]
                sun_files = [file for file in getFilelist(f'{temp_dump_fold}sun/', '.tif')]
                [os.remove(file) for file in bio_files]
                [os.remove(file) for file in sun_files]

            else:    
                [temp_files_vrt.remove(path) for path in [slope_path, aspect_path, thuenen_path]]
            

    
        # export the processed dates
        df = pd.DataFrame({
             'compdates': dateList_for_csv,
            })
        df.to_csv(f'{evap_outFolder}compdates.csv', index=False)

/data/Aldhani/users/potzschf/conda/envs/cds_era5/lib/python3.10/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


all dates of composites are the same :)
/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/LST_composites/maxLST/2019/
Skipping resampling: Angle shapes already matched (e.g., FORCE TIF).
Computing cosSZA, cosVZA and cosRAA
Scaling Sentinel-2 bands
Selecting sl2p input bands

--- Stacking Input Arrays ---
Done!
Generating sl2p input data flag
Run SL2P...
SL2P start: 2025-12-19 23:27:13.868969
SL2P end: 2025-12-19 23:27:14.422774
Generating sl2p output product flag
Done
Skipping resampling: Angle shapes already matched (e.g., FORCE TIF).
Computing cosSZA, cosVZA and cosRAA
Scaling Sentinel-2 bands
Selecting sl2p input bands

--- Stacking Input Arrays ---
Done!
Generating sl2p input data flag
Run SL2P...
SL2P start: 2025-12-19 23:27:15.145172
SL2P end: 2025-12-19 23:27:15.702152
Generating sl2p output product flag
Done
Skipping resampling: Angle shapes already matched (e.g., FORCE TIF).
Computing cosSZA, cosVZA and cosRAA
Scaling Sentinel-2 bands
Selecting sl2p input bands

--- Stackin

ERROR 5: GDALDataset::GetRasterBand(0) - Illegal band #



Adiabatic Sharpener will be applied
Adiabatic Sharpener will be applied
Barometric Sharpener will be applied


ERROR 5: GDALDataset::GetRasterBand(0) - Illegal band #



...

Iteration: 0, non-converged pixels: 2171964, max L diff: inf, total time: 0.003100, loop time: 0.003100


/home/potzschf/repos/other_repos/pyTSEB/pyTSEB/TSEB.py:1975: RuntimeWarning: invalid value encountered in divide
  L_diff = np.asarray(np.fabs(L - L_old) / np.fabs(L_old), dtype=np.float32)


Iteration: 1, non-converged pixels: 2171964, max L diff: inf, total time: 14.177795, loop time: 14.174695
Iteration: 2, non-converged pixels: 2171964, max L diff: 6.026939, total time: 35.960378, loop time: 21.782583
Iteration: 3, non-converged pixels: 2171964, max L diff: 4.561690, total time: 57.565880, loop time: 21.605502
Iteration: 4, non-converged pixels: 1674660, max L diff: 5.873160, total time: 78.427546, loop time: 20.861666
Iteration: 5, non-converged pixels: 1088329, max L diff: 2.717551, total time: 92.074610, loop time: 13.647063
Iteration: 6, non-converged pixels: 590979, max L diff: 0.710079, total time: 95.808742, loop time: 3.734132
Iteration: 7, non-converged pixels: 146303, max L diff: 1.753070, total time: 97.594015, loop time: 1.785273
Iteration: 8, non-converged pixels: 311, max L diff: 0.613005, total time: 98.390378, loop time: 0.796363
Iteration: 9, non-converged pixels: 29, max L diff: 1.244450, total time: 98.619356, loop time: 0.228978
Iteration: 10, non-co

ERROR 4: /data/Aldhani/eoagritwin/et/Sentinel3/LST/SharpEvap/Brandenburg/IntBio/sharpened/X0067Y0040/Values/maxLST_2019_April_02_mvwin15_cv0_regrat25_S2Masked_S2only_withoutLSTmask_X0067_Y0040.tif: No such file or directory


AttributeError: 'NoneType' object has no attribute 'GetRasterBand'

In [ ]:
sharped = sharpList[20]
comp = sharped.split('/')[-1].split('_')[0]
year = sharped.split('/')[-1].split('_')[1]
month = sharped.split('/')[-1].split('_')[2]
day = int(sharped.split('/')[-1].split('_')[3])
mvwin = sharped.split('/')[-1].split('_')[4]
cv = sharped.split('/')[-1].split('_')[5]
regrat = sharped.split('/')[-1].split('_')[6]
sharp = sharped.split('/')[-1].split('_')[8]
s2Mask = sharped.split('/')[-1].split('_')[7]
lstMask = sharped.split('/')[-1].split('_')[9]
tile = '_'.join(sharped.split('/')[-1].split('.')[0].split('_')[-2:])

   
if path_to_dem:
        bio_pars = [file for file in getFilelist(f'{temp_dump_fold}bio/', '.tif') if f'{year}_{month}_{day}.tif' in file]
else:
        bio_pars = False

In [ ]:
year=year
month=month
day=day
comp=comp
sharp=sharp
s2Mask=s2Mask
lstMask=lstMask
tile=tile
tempDir=trash_path
path_to_temp=temp_dump_fold
path_to_sharp=sharp_outFolder
mvwin=mvwin
cv=cv
regrat=regrat
evap_outFolder=evap_outFolder
printInterim=True
bio_pars=bio_pars

In [ ]:
storPath_c = f'{evap_outFolder}{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET_Canopy_calc.tif'
storPath_s = f'{evap_outFolder}{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET_Soil_calc.tif'

storPath_c_f = f'{evap_outFolder}{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET_Canopy_func.tif'
storPath_s_f = f'{evap_outFolder}{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET_Soil_func.tif'
    
# path to era5 raw data
era5_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/'
ssrd_mean_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/ssrd_mean_calc/'

# the DEM, SLOPE, ASPECT, LAT, LON will be used to sharpen some of the era5 variables (the the resolution of the DEM)
dem_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_FORCE_WARP.tif' # epsg 4326
slope_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/SLOPE_GER_FORCE_WARP.tif' # epsg 4326
aspect_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/ASPECT_GER_FORCE_WARP.tif' # epsg 4326
lat_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LAT_GER_FORCE_WARP.tif' # epsg 4326
lon_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LON_GER_FORCE_WARP.tif' # epsg 4326

# the geopotential is needed for the sharpening as well
geopot_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/geopotential/geopotential_low_res.tif' # epsg 4326


# path_base to sharpenend folder and S2_comp
sharp_pathbase = f'{path_to_sharp}Values/'
s2_pathbase = path_to_temp

# the LST acquisition time should determine which sharpened LST files are associatedto be processed (as they are associated with it)
LST_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/{year}/Daily_AcqTime_{comp}_{year}_{month}.tif' # epsg 4326

# the VZA at the time of LST acquisition is need
VZA_at_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/VZA/comp/{comp}/{year}/Daily_VZA_{comp}_{year}_{month}.tif' # epsg 4326

# sharpened LST
LST_file = f'{sharp_pathbase}{comp}_{year}_{month}_{day:02d}_{mvwin}_{cv}_{regrat}_{s2Mask}_{sharp}_{lstMask}_{tile}.tif' 
# for NDVI calculation (estimating LAI and others) and warping to S2 resolution, we use the S2 composite used for sharpening
# S2_file = [file for file in getFilelist(s2_pathbase, 'vrt', deep=False) if f'HIGHRES_{comp}_{year}_{month}_{day:02d}' in file][0]
S2_file = [file for file in getFilelist(s2_pathbase, 'vrt', deep=True) if 'S2' in file][0]

# find era5 file that matches the month of LST observation
valid_variables = sorted(list(dict.fromkeys(file.split('/')[-2] for file in getFilelist(era5_path, '.grib', deep=True) \
                                if not any(var in file for var in ['geopotential', 'total_column_water_vapour']))))

# get a list for those era5 files that match the year and month of the provided LST acquisition file
era5_path_list = find_grib_file(getFilelist(era5_path, '.grib', deep=True), LST_acq_file)
era5_path_list = [path for path in era5_path_list if any(variable in path for variable in valid_variables)] # era5 are epsg 4326 and still will be after warping to doy
temp_pressure_checker(era5_path_list)

# warp datasets needed for calculations to the spatial extent of the sharpened LST
LST_acq_spatial_sub = warp_raster_to_reference(source_path=LST_acq_file, reference_path=S2_file, output_path='MEM', resampling='near')
VZA_at_acq_file_sub = warp_raster_to_reference(source_path=VZA_at_acq_file, reference_path=S2_file, output_path='MEM', resampling='near')
dem_sub = warp_raster_to_reference(source_path=dem_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
slope_sub  = warp_raster_to_reference(source_path=slope_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
aspect_sub = warp_raster_to_reference(source_path=aspect_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
lat_sub = warp_raster_to_reference(source_path=lat_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
lon_sub = warp_raster_to_reference(source_path=lon_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
geopot_sub = warp_raster_to_reference(source_path=geopot_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')

# print if needed
if printInterim:
    if path_to_temp.endswith('/'):
        temp_path_sub = f'{path_to_temp}evap_interim/'
    else:
        temp_path_sub = f'{path_to_temp}/evap_interim/'

    os.makedirs(temp_path_sub, exist_ok=True)

    id_tag = f'{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}'
    LST_acq_spatial_arr = LST_acq_spatial_sub.GetRasterBand(day).ReadAsArray()
    VZA_at_acq_file_arr = VZA_at_acq_file_sub.GetRasterBand(day).ReadAsArray()
    dem_arr= dem_sub.GetRasterBand(1).ReadAsArray()
    slope_arr = slope_sub.GetRasterBand(1).ReadAsArray()
    aspect_arr = aspect_sub.GetRasterBand(1).ReadAsArray()
    lat_arr = lat_sub.GetRasterBand(1).ReadAsArray()
    lon_arr = lon_sub.GetRasterBand(1).ReadAsArray()
    geopot_arr = geopot_sub.GetRasterBand(1).ReadAsArray()


    npTOdisk(LST_acq_spatial_arr, S2_file, f'{temp_path_sub}LST_acq_spatial_sub_{id_tag}.tif', bands = 1, d_type=gdal.GDT_Int64)
    npTOdisk(VZA_at_acq_file_arr, S2_file, f'{temp_path_sub}VZA_at_acq_file_sub_{id_tag}.tif', bands = 1)
    npTOdisk(dem_arr, S2_file, f'{temp_path_sub}dem_sub_{id_tag}.tif', bands = 1)
    npTOdisk(slope_arr, S2_file, f'{temp_path_sub}slope_sub_{id_tag}.tif', bands = 1)
    npTOdisk(aspect_arr, S2_file, f'{temp_path_sub}aspect_sub_{id_tag}.tif', bands = 1)
    npTOdisk(lat_arr, S2_file, f'{temp_path_sub}lat_sub_{id_tag}.tif', bands = 1)
    npTOdisk(lon_arr, S2_file, f'{temp_path_sub}lon_sub_{id_tag}.tif', bands = 1)
    npTOdisk(geopot_arr, S2_file, f'{temp_path_sub}geopot_sub_{id_tag}.tif', bands = 1)


In [ ]:
# load the era5 variable into cache at LST resolution and read-in the modelled times (one time step per band)
for path in era5_path_list:
    # print(f'processing {path}')
    # check if DEM sharpener needs to be applied
    if '100m_u_component_of_wind' in path:
        # do the warping without sharpening
        try:
            wind100_u = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)
        except Exception as e:
            with open(f'{tempDir}ERROR_{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET.log', 'a') as f:
                f.write(f'{e}')
            continue

    elif '100m_v_component_of_wind' in path:
            # do the warping without sharpening
        try:
            wind100_v = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)
        except Exception as e:
            with open(f'{tempDir}ERROR_{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET.log', 'a') as f:
                f.write(f'{e}')
            continue
    # elif 'geopotential' in path:
    #     # do the warping without sharpening
    #     geopot = get_warped_ERA5_at_doy(path_to_era_grib=path, lst_acq_file=LST_acq_file, doy=day)

    elif 'downward' in path: # terrain correction included
        try:
            ssrd, szenith, sazimuth, ssrd_nc, ssrd_mean_func = get_ssrdsc_warped_and_corrected_at_doy(path_to_ssrdsc_grib=path, reference_path=LST_acq_spatial_sub, 
                                                                            lst_acq_file=LST_acq_spatial_sub, doy=day, 
                                                                            slope_path=slope_sub,
                                                                            aspect_path=aspect_sub,
                                                                            dem_path=dem_sub,
                                                                            lat_path=lat_sub,
                                                                            lon_path=lon_sub)
        except Exception as e:
            with open(f'{tempDir}ERROR_{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET.log', 'a') as f:
                f.write(f'{e}')
            continue
        
    elif '2m_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        try:
            air_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                            lst_acq_file=LST_acq_spatial_sub, doy=day,
                                            sharp_blendheight=100,
                                            sharp_DEM=dem_sub,
                                            sharp_geopot=geopot_sub,
                                            sharp_rate=STANDARD_ADIABAT,
                                            sharpener='adiabatic')
        except Exception as e:
            with open(f'{tempDir}ERROR_{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET.log', 'a') as f:
                f.write(f'{e}')
            continue
        
    elif '2m_dewpoint_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        try:
            dew_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                            lst_acq_file=LST_acq_spatial_sub, doy=day,
                                            sharp_blendheight=100,
                                            sharp_DEM=dem_sub,
                                            sharp_geopot=geopot_sub,
                                            sharp_rate=MOIST_ADIABAT,
                                            sharpener='adiabatic')
        except Exception as e:
            with open(f'{tempDir}ERROR_{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET.log', 'a') as f:
                f.write(f'{e}')
            continue
        
    else: 
        # do warping with DEM sharpening only
        # sanity check
        if not 'surface_pressure' in path:
            raise ValueError('There is and unattended ERA5 variable in the loop - CHECK!!!!')
        else:
            try:
                sp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                            lst_acq_file=LST_acq_spatial_sub, doy=day,
                                            sharp_DEM=dem_sub,
                                            sharp_blendheight=100,
                                            sharp_geopot=geopot_sub,
                                            sharp_temp=air_temp,
                                            sharpener='barometric') / 100
            except Exception as e:
                with open(f'{tempDir}ERROR_{comp}_{year}_{month}_{day}_{mvwin}_{cv}_{regrat}_{lstMask}_{s2Mask}_{sharp}_{tile}_ET.log', 'a') as f:
                    f.write(f'{e}')
                continue

In [ ]:
wind_speed_20 = calc_wind_speed(wind100_u, wind100_v) # check wind_u

ds = gdal.Open(f'{ssrd_mean_path}surface_solar_radiation_downward_clear_sky_{year}_{int(MONTH_TO_02D[month])}')
ssrd_mean = ds.GetRasterBand(day).ReadAsArray() / 3600

ssrd_mean_calc_20 = warp_np_to_reference(ssrd_mean, f'{ssrd_mean_path}surface_solar_radiation_downward_clear_sky_{year}_{int(MONTH_TO_02D[month])}', LST_file) # check this too!!!!!
ssrd_mean_func_20 = ssrd_mean_func
ssrd_20 = ssrd
air_temp_20 = air_temp
dew_temp_20 = dew_temp
sp_20 = sp
szenith_20 = szenith
sazimuth_20 = sazimuth

# calculate windspeed

# load vza
vza_ds = VZA_at_acq_file_sub
vza_20 = vza_ds.GetRasterBand(day).ReadAsArray()

# load sharpened LST
lst_ds = gdal.Open(LST_file)
lst_20 =lst_ds.GetRasterBand(1).ReadAsArray()

del wind100_u, wind100_v, ssrd, air_temp, dew_temp, sp, szenith, sazimuth, ssrd_mean, ssrd_nc, ssrd_mean_func


if printInterim:
    npTOdisk(ssrd_mean_calc_20, LST_file, f'{temp_path_sub}SSRD_mean_calc_{id_tag}.tif', bands = 1)
    npTOdisk(ssrd_mean_func_20, LST_file, f'{temp_path_sub}SSRD_mean_func_{id_tag}.tif', bands = 1)
    npTOdisk(ssrd_20, LST_file, f'{temp_path_sub}SSRD_{id_tag}.tif', bands = 1)
    npTOdisk(air_temp_20, LST_file, f'{temp_path_sub}TEMP_{id_tag}.tif', bands = 1)
    npTOdisk(dew_temp_20, LST_file, f'{temp_path_sub}DEW_{id_tag}.tif', bands = 1)
    npTOdisk(sp_20, LST_file, f'{temp_path_sub}SP_{id_tag}.tif', bands = 1)
    npTOdisk(szenith_20, LST_file, f'{temp_path_sub}ZEN_{id_tag}.tif', bands = 1)
    npTOdisk(sazimuth_20, LST_file, f'{temp_path_sub}AZI_{id_tag}.tif', bands = 1)
    npTOdisk(wind_speed_20, LST_file, f'{temp_path_sub}WSPEED_{id_tag}.tif', bands = 1)
    npTOdisk(lst_20, LST_file, f'{temp_path_sub}LST_{id_tag}.tif', bands = 1)
    npTOdisk(vza_20, LST_file, f'{temp_path_sub}VZA_{id_tag}.tif', bands = 1)

condition = (air_temp_20 > 0) & (dew_temp_20 > 0)  & (sp_20 > 0) & (szenith_20 > 0) & (sazimuth_20 > 0) & (wind_speed_20 > 0) & (lst_20 > 0) & (vza_20 > 0)
ssrd_20[~condition] = np.nan
ssrd_mean_calc_20[~condition] = np.nan
ssrd_mean_func_20[~condition] = np.nan
air_temp_20[~condition] = np.nan
dew_temp_20[~condition] = np.nan
sp_20[~condition] = np.nan
szenith_20[~condition] = np.nan
sazimuth_20[~condition] = np.nan
wind_speed_20[~condition] = np.nan
# lst_20 = np.ma.masked_where(~condition, lst_20)
# vza_20 = np.ma.masked_where(~condition, vza_20)
lst_20[~condition] = np.nan
vza_20[~condition] = np.nan

if printInterim:
    npTOdisk(ssrd_mean_calc_20, LST_file, f'{temp_path_sub}SSRD_mean_calc_masked_{id_tag}.tif', bands = 1)
    npTOdisk(ssrd_mean_func_20, LST_file, f'{temp_path_sub}SSRD_mean_func_masked_{id_tag}.tif', bands = 1)
    npTOdisk(ssrd_20, LST_file, f'{temp_path_sub}SSRD_masked_{id_tag}.tif', bands = 1)
    npTOdisk(air_temp_20, LST_file, f'{temp_path_sub}TEMP_masked_{id_tag}.tif', bands = 1)
    npTOdisk(dew_temp_20, LST_file, f'{temp_path_sub}DEW_masked_{id_tag}.tif', bands = 1)
    npTOdisk(sp_20, LST_file, f'{temp_path_sub}SP_masked_{id_tag}.tif', bands = 1)
    npTOdisk(szenith_20, LST_file, f'{temp_path_sub}ZEN_masked_{id_tag}.tif', bands = 1)
    npTOdisk(sazimuth_20, LST_file, f'{temp_path_sub}AZI_masked_{id_tag}.tif', bands = 1)
    npTOdisk(wind_speed_20, LST_file, f'{temp_path_sub}WSPEED_masked_{id_tag}.tif', bands = 1)
    npTOdisk(lst_20, LST_file, f'{temp_path_sub}LST_masked_{id_tag}.tif', bands = 1)
    npTOdisk(vza_20, LST_file, f'{temp_path_sub}VZA_masked_{id_tag}.tif', bands = 1)


In [ ]:


# def canopy_reflectance(rho_leaf, tau_leaf, PAI, mu, rho_soil):
#     """
#     Simple spherical leaf distribution two-stream approximation.
#     rho_leaf, tau_leaf: leaf optics (arrays)
#     PAI: plant area index (array)
#     mu: cos(theta) (array)
#     rho_soil: scalar
#     """
#     # Clip inputs
#     PAI_safe = np.clip(PAI, 0.0, 15.0)
#     mu_safe = np.clip(mu, 0.01, 1.0)  # avoid division by zero

#     omega = rho_leaf + tau_leaf  # single scattering albedo
#     k = 0.5 / mu_safe            # extinction coefficient
#     T = np.exp(-k * PAI_safe)    # gap fraction

#     rho_canopy = omega * (1 - T) / (1 + T) + rho_soil * T

#     # Handle NaN or inf
#     rho_canopy = np.nan_to_num(rho_canopy, nan=rho_soil, posinf=1.0, neginf=0.0)
#     T = np.nan_to_num(T, nan=0.0, posinf=1.0, neginf=0.0)

#     # Clip physically valid range
#     rho_canopy = np.clip(rho_canopy, 0.0, 1.0)
#     T = np.clip(T, 0.0, 1.0)

#     return rho_canopy, T

# def surface_albedo(Cab, Cw, PAI, theta, rho_soil_vis=0.15, rho_soil_nir=0.25):
#     """
#     Compute broadband albedo and transmission.
#     theta: solar zenith angle in radians
#     """
#     mu = np.cos(theta)
#     mu = np.clip(mu, 0.01, 1.0)

#     # Leaf optics
#     rho_l_vis, tau_l_vis = leaf_optics_vis(Cab)
#     rho_l_nir, tau_l_nir = leaf_optics_nir(Cw)

#     # Canopy reflectance
#     rho_vis, T_vis = canopy_reflectance(rho_l_vis, tau_l_vis, PAI, mu, rho_soil_vis)
#     rho_nir, T_nir = canopy_reflectance(rho_l_nir, tau_l_nir, PAI, mu, rho_soil_nir)

#     # Broadband albedo (average VIS + NIR)
#     albedo_sw = 0.5 * (rho_vis + rho_nir)

#     # Ensure no NaN/inf and clip
#     albedo_sw = np.nan_to_num(albedo_sw, nan=0.2, posinf=1.0, neginf=0.0)
#     albedo_sw = np.clip(albedo_sw, 0.0, 1.0)

#     T_vis = np.clip(T_vis, 0.0, 1.0)
#     T_nir = np.clip(T_nir, 0.0, 1.0)

#     return albedo_sw, T_vis, T_nir

# def net_shortwave_radiation(Rs, albedo_sw, T_vis, T_nir):
#     """
#     Compute net shortwave radiation on canopy and soil.
#     Rs: incoming shortwave (W/m²)
#     """
#     # Clip Rs
#     Rs_safe = np.clip(Rs, 0.0, None)

#     # Net absorbed at surface
#     Rns = (1 - albedo_sw) * Rs_safe
#     Rns = np.nan_to_num(Rns, nan=0.0, posinf=Rs_safe, neginf=0.0)

#     # Soil receives transmitted fraction
#     Rns_soil = Rns * 0.5 * (T_vis + T_nir)
#     Rns_soil = np.nan_to_num(Rns_soil, nan=0.0, posinf=Rs_safe, neginf=0.0)

#     # Canopy gets remainder
#     Rns_canopy = Rns - Rns_soil
#     Rns_canopy = np.clip(Rns_canopy, 0.0, None)

#     return Rns_canopy, Rns_soil


In [ ]:
albedo, ccc, cwc, lai, fapar, fcover = read_biophys(bio_pars)
theta = np.deg2rad(szenith_20)
theta_safe = np.clip(theta, 0.05, 1.0)

lai_min = 0.1
valid = lai > lai_min

Cab = np.full_like(lai, np.nan, dtype=float)
Cw  = np.full_like(lai, np.nan, dtype=float)

Cab[valid] = ccc[valid] / lai[valid]
Cw[valid]  = cwc[valid] / lai[valid]


In [ ]:
# calculate the NDVI from the S2 composite (following formula from force --> bandnames: (NIR - RED) / (NIR + RED))
S2_ds = gdal.Open(S2_file)
for idx, bname in enumerate(getBandNames(S2_file)):
    if bname == 'RED':
        red = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    elif bname == 'BNR':
        nir = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    else:
        continue

ndvi_20 = (nir - red) / (nir + red)
ndvi_20_ma = np.where(ndvi_20 < 0, np.nan, ndvi_20)
# ndvi_20_ma = np.ma.masked_invalid(ndvi_20)
# ndvi_20_ma = np.ma.masked_where(ndvi_20_ma < 0, ndvi_20_ma)

fg, FIPAR, PAI = compute_fg_fipar_pai(
    LAI=lai,
    FAPAR=fapar,
    theta=theta)

# LAI_np = 0.57*np.exp(2.33*ndvi_20)
# LAI_pos = np.where(LAI_np < 0, np.nan, LAI_np)
LAI_np = lai
LAI_pos = np.where(LAI_np < 0, np.nan, LAI_np)
# estimate canopy height from estimated LAI
hc = hc_from_lai(LAI_pos, hc_max = 1.2, lai_max = np.nanmax(LAI_np), hc_min=0)


In [ ]:

# estimate long wave irradiance
ea = meteo_utils.calc_vapor_pressure(T_K=dew_temp_20)
L_dn = calc_longwave_irradiance(ea = ea, t_a_k = air_temp_20, p = sp_20, z_T = 100, h_C = hc) # ## does that make sense with the 100m!!!!!!!!!!!!!!!!!!!
d_0_0 = resistances.calc_d_0(h_C=hc)
z_0 = resistances.calc_z_0M(h_C=hc)


# calculate shortwave radiation of soil and canopy

# difvis, difnir, fvis, fnir = net_radiation.calc_difuse_ratio(S_dn = ssrd_20, sza = np.nanmean(szenith_20))
ssrd_1d = ssrd_20.ravel()
sza_1d  = szenith_20.ravel()

press_1d = 1013.25 * np.exp(-dem_arr.ravel() / 8434.5)
difvis, difnir, fvis, fnir = net_radiation.calc_difuse_ratio(S_dn=ssrd_1d,sza=sza_1d,press=press_1d)

difvis = difvis.reshape(ssrd_20.shape)
difnir = difnir.reshape(ssrd_20.shape)
fvis   = fvis.reshape(ssrd_20.shape)
fnir   = fnir.reshape(ssrd_20.shape)


skyl = difvis * fvis + difnir * fnir
S_dn_dir = ssrd_20 * (1.0 - skyl)
S_dn_dif = ssrd_20 * skyl

# Leaf spectral properties:{rho_vis_C: visible reflectance, tau_vis_C: visible transmittance, rho_nir_C: NIR reflectance, tau_nir_C: NIR transmittance}
Cab = ccc / np.maximum(lai, 1e-6)
Cw  = cwc / np.maximum(lai, 1e-6)
rho_vis_C, tau_vis_C = leaf_optics_vis(Cab)
rho_nir_C, tau_nir_C = leaf_optics_nir(Cw)
# rho_vis_C=np.full(LAI_pos.shape, 0.05, np.float32)
# tau_vis_C=np.full(LAI_pos.shape, 0.08, np.float32)
# rho_nir_C=np.full(LAI_pos.shape, 0.32, np.float32)
# tau_nir_C=np.full(LAI_pos.shape, 0.33, np.float32) 

# Soil spectral properties:{rho_vis_S: visible reflectance, rho_nir_S: NIR reflectance}
rho_vis_S=np.full(LAI_pos.shape, 0.07, np.float32)
rho_nir_S=np.full(LAI_pos.shape, 0.25, np.float32)

# F = local LAI
F = LAI_pos / 1
# calculate clumping index
f_c = np.clip(fcover, 0.05, 0.99)
Omega0 = clumping_index.calc_omega0_Kustas(LAI = LAI_np, f_C = f_c, x_LAD=1)
Omega = clumping_index.calc_omega_Kustas(Omega0, szenith_20)
LAI_eff = F * Omega

Sn_C, Sn_S = net_radiation.calc_Sn_Campbell(lai = LAI_pos, sza = szenith_20, S_dn_dir = S_dn_dir, S_dn_dif = S_dn_dif, fvis = fvis,
                                    fnir = fnir, rho_leaf_vis = rho_vis_C, tau_leaf_vis = tau_vis_C, rho_leaf_nir = rho_nir_C, 
                                    tau_leaf_nir = tau_nir_C, rsoilv = rho_vis_S, rsoiln = rho_nir_S, x_LAD=1, LAI_eff=LAI_eff)

# calculate other roughness stuff

landC = np.full(LAI_pos.shape, 11, dtype=np.int16)
w_C = np.ones_like(LAI_pos, dtype=np.float32)
z_0M, d = resistances.calc_roughness(LAI=LAI_pos, h_C=hc, w_C=w_C, landcover=landC, f_c=f_c)
# fg = calc_fg_gutman(ndvi = ndvi_20_ma, ndvi_min = np.nanmin(ndvi_20), ndvi_max = np.nanmax(ndvi_20))

if printInterim:
    npTOdisk(ndvi_20, LST_file, f'{temp_path_sub}ndvi_{id_tag}.tif')
    npTOdisk(ndvi_20_ma, LST_file, f'{temp_path_sub}ndvi_pos_{id_tag}.tif')
    npTOdisk(LAI_np, LST_file, f'{temp_path_sub}LAI_{id_tag}.tif')
    npTOdisk(LAI_pos, LST_file, f'{temp_path_sub}LAI_pos_{id_tag}.tif')
    npTOdisk(hc, LST_file, f'{temp_path_sub}canopy_height_{id_tag}.tif')
    npTOdisk(ea, LST_file, f'{temp_path_sub}vapor_pressure_{id_tag}.tif')
    npTOdisk(L_dn, LST_file, f'{temp_path_sub}longwave_radiation_{id_tag}.tif')
    npTOdisk(d_0_0, LST_file, f'{temp_path_sub}resistanceD_{id_tag}.tif')
    npTOdisk(z_0, LST_file, f'{temp_path_sub}resistanceZ_{id_tag}.tif')

    npTOdisk(Omega0, LST_file, f'{temp_path_sub}Omega0_{id_tag}.tif')
    npTOdisk(Omega, LST_file, f'{temp_path_sub}Omega_{id_tag}.tif')
    npTOdisk(Sn_C, LST_file, f'{temp_path_sub}Sn_C_{id_tag}.tif')
    npTOdisk(Sn_S, LST_file, f'{temp_path_sub}Sn_S_{id_tag}.tif')
    npTOdisk(fg, LST_file, f'{temp_path_sub}fg_{id_tag}.tif')

emis_C = 0.98
emis_S = 0.95
h_C = hc 
z_u = 100
z_T = 100

output = TSEB.TSEB_PT(lst_20, vza_20, air_temp_20, wind_speed_20, ea, sp_20, Sn_C, Sn_S, L_dn, LAI_pos, h_C, emis_C, emis_S, 
z_0M, d, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None, f_g=fg,
kB=0.0, massman_profile=None, verbose=True)

for stori, ssrd_ras in zip([[storPath_c, storPath_s],[storPath_c_f, storPath_s_f]], [ssrd_mean_calc_20, ssrd_mean_func_20]):
    
    le_c = output[6]/ssrd_20
    heat_latent_scaled_c = ssrd_ras * le_c
    et_daily_c = TSEB.met.flux_2_evaporation(heat_latent_scaled_c, t_k=air_temp_20, time_domain=24)

    le_s = output[8]/ssrd_20
    heat_latent_scaled_s = ssrd_ras * le_s
    et_daily_s = TSEB.met.flux_2_evaporation(heat_latent_scaled_s, t_k=air_temp_20, time_domain=24)

    npTOdisk(et_daily_c, LST_file, stori[0])
    npTOdisk(et_daily_s, LST_file, stori[1])